In [ ]:
# libraries
library(magrittr)
library(raster)
library(sf)
library(terra)
library(rgeos)

In [ ]:
# Load in crowns - needed for both the single loop example, and the loop further down

Sep_crowns=st_read("~/lustre_scratch/sepilok_data/thomas_sepilok_crowns/thomas_no_intersection.shp")


In [ ]:
### Let's test our code on a single tile

# set the origin of our test tile
# JB: can't this be done automatically?
x_tile = 603330
y_tile = 647470

# set the boundaries - we are going to remove the buffer from this test tile

xmin= x_tile; xmax= x_tile + 200; ymin= y_tile; ymax= y_tile + 200

# read in the tiled tiff

Sep_2014_RGB = raster::stack(paste0("~/lustre_scratch/sepilok_data/sepilok_2014_rgb/RCD105_MA14_21_orthomosaic_20141023_reprojected_full_res.tif"))

# Crop to the extent of the tile
### Possibly able to remove this? It is already cropped right...
### Would only be needed if we were reading in the whole tiff of all of Sepilok

tile_RGB=raster::crop(Sep_2014_RGB,extent(xmin,xmax,ymin,ymax))

# Create a polygon of the tile outline 
tile_poly= list(rbind(c(xmin, ymin), c(xmax, ymin), c(xmax,ymax),c(xmin, ymax),c(xmin, ymin))) %>%
  st_polygon() %>% st_sfc() %>% st_set_crs(32650) #%>% st_geometry

# Crop crown polygons to the tile polygon
tile_crowns=st_intersection(Sep_crowns,tile_poly) # bbox(crowns) != bbox(RGB)

# Plot the tile
plotRGB(tile_RGB,axes=TRUE, scale = 75000)
plot(tile_crowns,axes=TRUE,add=TRUE)

In [ ]:
### ignore for now
### messing here with saving - should do this in another script...

### st_write(tile_crowns,paste0("~/lustre_scratch/paracou_data/fresh_2019_10cm_crowns_",x_tile,"_",y_tile,".shp"))
    
                                                                            

In [ ]:
# save the tiff as a png with the following three commands (dev.off() is key)

png(paste0("~/lustre_scratch/sepilok_data/all_sepilok_tiles_hi_res/tile_", x_tile,"_", y_tile,".png", sep=''), 
                                                                    width=ncol(tile_RGB), height=nrow(tile_RGB))
plotRGB(tile_RGB, maxpixels = ncell(tile_RGB),   axes = FALSE, scale = 75000)
dev.off()

In [ ]:
# load the png we just saved

load_tile_RGB=raster::stack(paste0("~/lustre_scratch/sepilok_data/sepilok_2014_rgb_tiles_pngs/tile_", 
                                                                            x_tile,"_", y_tile,".png"))
# Translate the tree crowns to coordinates to match new space
tile_crowns_00=st_geometry(tile_crowns)-c(xmin,ymin)
tile_crowns_new_coords=tile_crowns_00*10 # multiply crown coords by 10 because the resolution of the RGB is...10cm, not 8.64 as specified...
    
# check it overlaps with the RGB in the new space
plotRGB(load_tile_RGB,axes=TRUE)
plot(tile_crowns_new_coords,axes=TRUE,add=TRUE)

In [ ]:
# Save crowns as geojson 
st_write(tile_crowns_new_coords,paste0("~/lustre_scratch/sepilok_data/sepilok_2014_rgb_tiled_crown_geojsons/Sep_2014_RGB_crowns_",
                                                                                                x_tile,"_",y_tile,".geojson"))
    
# Load data back in and see if it overlaps in the new space
load_tile_crowns=st_read(paste0("~/lustre_scratch/sepilok_data/sepilok_2014_rgb_tiled_crown_geojsons/Sep_2014_RGB_crowns_",
                                                                                            x_tile,"_",y_tile,".geojson"))

# plot the RGB and geojson to check it everything is still in the right place!
# we have just saved it, and have then read it in, so should be fine
plotRGB(load_tile_RGB,axes=TRUE)
plot(load_tile_crowns,axes=TRUE,add=TRUE)

In [ ]:
### Now let's do this as a loop over all our tiles (a lot!!)

#c = 1

### Can loop over the file list here...
### Or go over the crowns - that is our limiting factor...
### are there manual crowns in this space? If length of crowns = 0, skip, if length of crowns != 0, then do!

for (x_tile in seq(602300,603400,100)){ #Approximate extent of the RGB raster
  
  for (y_tile in seq(646200,647900,100)){
    
    xmin=x_tile; xmax=x_tile+100 # 100 m tiles with a 20m buffer
    ymin=y_tile; ymax=y_tile+100
    
      
    # wrap in try or tryCatch to catch the error and essentially skip over it, if the tile does not exist
    # we want to try to use tryCatch to let the loop carry on despite errors
    # TBC
      
    # crowns in tile
    tile_poly= list(rbind(c(xmin, ymin), c(xmax, ymin), c(xmax,ymax),c(xmin, ymax),c(xmin, ymin))) %>%
      st_polygon() %>% st_sfc() %>% st_set_crs(32650)
    tile_crowns=st_intersection(Sep_crowns,tile_poly)
    
    if (nrow(tile_crowns) != 0){
        
    Sep_2014_RGB_tile = raster::stack(paste0("~/lustre_scratch/sepilok_data/sepilok_2014_rgb/RCD105_MA14_21_orthomosaic_20141023_reprojected_full_res.tif"))
    #Sep_2014_RGB_tile = raster::stack(paste0("~/lustre_scratch/sepilok_data/sepilok_2014_rgb_tiles/Sep_2014_RGB_",x_tile,"_",y_tile,".tif"))
      
    # this is probably unnecessary - the tile already has this extent at this point
    tile_RGB=raster::crop(Sep_2014_RGB_tile,extent(xmin,xmax,ymin,ymax))
      
    # Make polygon around the extent of the tile for cropping the crowns
    #tile_poly= list(rbind(c(xmin, ymin), c(xmax, ymin), c(xmax,ymax),c(xmin, ymax),c(xmin, ymin))) %>%
    #  st_polygon() %>% st_sfc() %>% st_set_crs(32650)
    
    # put this first
    #tile_crowns=st_intersection(Sep_crowns,tile_poly)
    
    plotRGB(tile_RGB,axes=TRUE, scale=70000)
    #plot(tile_crowns,axes=TRUE,add=TRUE)
    
    # save the tiff as a png here
    png(paste0("~/lustre_scratch/sepilok_data/thomas_sepilok_tiles_hi_res/tile_", x_tile,"_", y_tile,".png", sep=''), 
                                                                        width=ncol(tile_RGB), height=nrow(tile_RGB))
    plotRGB(tile_RGB, maxpixels = ncell(tile_RGB), axes = FALSE, scale = 70000)
    dev.off()
    
    # read in this png
    load_tile_RGB=raster::stack(paste0("~/lustre_scratch/sepilok_data/thomas_sepilok_tiles_hi_res/tile_", 
                                                                                x_tile,"_", y_tile,".png"))
                            
    # Translate the tree crowns to coordinates to match new space
    tile_crowns_00=st_geometry(tile_crowns)-c(xmin,ymin)
    tile_crowns_new_coords=tile_crowns_00*10 # multiply crown coords by 2 because the resolution of the RGB is 50cm
    
    # check it overlaps with the RGB in the new space
    plotRGB(load_tile_RGB,axes=TRUE)
    plot(tile_crowns_new_coords,axes=TRUE,add=TRUE)
    
    # Save crowns as geojson 
    st_write(tile_crowns_new_coords,paste0("~/lustre_scratch/sepilok_data/thomas_sepilok_tiled_manual_crowns/Sep_2014_RGB_crowns_",
                                                                                                x_tile,"_",y_tile,".geojson"),
                                                                                                append=FALSE)
    
    # Load data back in and see if it overlaps in the new space
    #load_tile_crowns=st_read(paste0("~/lustre_scratch/sepilok_data/sepilok_2014_rgb_tiled_crown_geojsons/Sep_2014_RGB_crowns_",
    #                                                                                        x_tile,"_",y_tile,".geojson"))
    
    #plotRGB(load_tile_RGB,axes=TRUE)
    #plot(load_tile_crowns,axes=TRUE,add=TRUE)
    
    #c=c+1
    #print(c)
    }}
}


In [ ]:
### here I am adding the property: imgPath to the geojsons, so the geojsons are linked to a png

for (x_tile in seq(602000,607900,100)){ #Approximate extent of the RGB raster
  
  for (y_tile in seq(646300,648600,100)){
      
      tile_poly= list(rbind(c(xmin, ymin), c(xmax, ymin), c(xmax,ymax),c(xmin, ymax),c(xmin, ymin))) %>%
          st_polygon() %>% st_sfc() %>% st_set_crs(32650)
      tile_crowns=st_intersection(Sep_crowns,tile_poly)
      
      if (nrow(tile_crowns) != 0){
      
      load_tile_crowns=st_read(paste0("~/lustre_scratch/sepilok_data/sepilok_2014_rgb_tiled_crown_geojsons/Sep_2014_RGB_crowns_",
                                                                                            x_tile,"_",y_tile,".geojson"))
      x <-'{ "imagePath": "tile_", x_tile,"_", y_tile,".png""}'
     (y <- linestring(x))
      
      }
  }
}
          
          